In [1]:
import pandas as pd
import numpy as np
import us
from geopy.geocoders import Nominatim
from sklearn.neighbors import NearestNeighbors

In [2]:
demographicData = pd.read_csv("Data/Demographics/ACSDP1Y2019.csv", skiprows=[0])
incomeData = pd.read_csv("Data/Income/ACSST1Y2019.csv", skiprows=[0])
urbanData = pd.read_csv("2010CensusUrbanRural.csv", skiprows=[0])


In [3]:
demographicData

,id,Geographic Area Name,Estimate!!SEX AND AGE!!Total population,Margin of Error!!SEX AND AGE!!Total population,Percent!!SEX AND AGE!!Total population,Percent Margin of Error!!SEX AND AGE!!Total population,Estimate!!SEX AND AGE!!Total population!!Male,Margin of Error!!SEX AND AGE!!Total population!!Male,Percent!!SEX AND AGE!!Total population!!Male,Percent Margin of Error!!SEX AND AGE!!Total population!!Male,...,"Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Male","Estimate!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female","Percent Margin of Error!!CITIZEN, VOTING AGE POPULATION!!Citizen, 18 and over population!!Female"
0,0400000US01,Alabama,4903185,*****,4903185,(X),2369611,5813,48.3,0.1,...,3731336,(X),1768386,4711,47.4,0.1,1962950,4023,52.6,0.1
1,0400000US02,Alaska,731545,*****,731545,(X),380433,2130,52.0,0.3,...,533151,(X),278479,2343,52.2,0.3,254672,2031,47.8,0.3
2,0400000US04,Arizona,7278717,*****,7278717,(X),3620935,2892,49.7,0.1,...,5137474,(X),2533931,10100,49.3,0.1,2603543,9762,50.7,0.1
3,0400000US05,Arkansas,3017804,*****,3017804,(X),1474705,4896,48.9,0.2,...,2235415,(X),1075762,4729,48.1,0.2,1159653,4437,51.9,0.2
4,0400000US06,California,39512223,*****,39512223,(X),19640794,6786,49.7,0.1,...,26032160,(X),12786049,32037,49.1,0.1,13246111,27222,50.9,0.1
5,0400000US08,Colorado,5758736,*****,5758736,(X),2904955,4852,50.4,0.1,...,4244210,(X),2128648,8907,50.2,0.1,2115562,6315,49.8,0.1
6,0400000US09,Connecticut,3565287,*****,3565287,(X),1743777,2405,48.9,0.1,...,2619474,(X),1253707,6831,47.9,0.2,1365767,6310,52.1,0.2
7,0400000US10,Delaware,973764,*****,973764,(X),470288,1336,48.3,0.1,...,725178,(X),344221,2358,47.5,0.2,380957,2594,52.5,0.2
8,0400000US11,District of Columbia,705749,*****,705749,(X),334355,513,47.4,0.1,...,536768,(X),249106,2526,46.4,0.4,287662,2811,53.6,0.4
9,0400000US12,Florida,21477737,*****,21477737,(X),10498005,7711,48.9,0.1,...,15507315,(X),7459575,23711,48.1,0.1,8047740,19367,51.9,0.1


In [4]:
margins = [column for column in demographicData.columns if "Margin" in column]

In [5]:
demographicData = demographicData.drop(columns = margins)

In [6]:
geolocator = Nominatim(user_agent = "MDST-election-prediction")

In [7]:
lat = {}
long = {}
for index, state in demographicData["Geographic Area Name"].iteritems():
    loc = geolocator.geocode(state)
    lat[state] = loc.latitude
    long[state] = loc.longitude

In [8]:
demographicData["Longitude"] = demographicData["Geographic Area Name"].map(long)

In [9]:
demographicData["Latitude"] = demographicData["Geographic Area Name"].map(lat)

In [10]:
feat_columns = ["Estimate!!SEX AND AGE!!Total population!!Median age (years)",
                "Percent!!RACE!!Total population!!One race!!Black or African American",
               "Percent!!RACE!!Total population!!One race!!American Indian and Alaska Native",
               "Percent!!RACE!!Total population!!One race!!Asian",
               "Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)",
               "Latitude",
               "Longitude"]

In [11]:
demoFeatData = demographicData[feat_columns]

In [12]:
educationData = pd.read_csv("Data/ACSST1Y2019.S1501_data_with_overlays_2020-10-22T140417.csv",skiprows=([0]))

In [13]:
margins = [column for column in educationData.columns if "Margin" in column]

In [14]:
educationData = educationData.drop(columns = margins)
educationData

,id,Geographic Area Name,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Less than high school graduate,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!High school graduate (includes equivalency),Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Some college or associate's degree,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Less than 9th grade,"Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!9th to 12th grade, no diploma",...,Estimate!!Percent Female!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!Less than high school graduate,Estimate!!Percent Female!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!High school graduate (includes equivalency),Estimate!!Percent Female!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!Some college or associate's degree,Estimate!!Percent Female!!POVERTY RATE FOR THE POPULATION 25 YEARS AND OVER FOR WHOM POVERTY STATUS IS DETERMINED BY EDUCATIONAL ATTAINMENT LEVEL!!Bachelor's degree or higher,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Less than high school graduate,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!High school graduate (includes equivalency),Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Some college or associate's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Bachelor's degree,Estimate!!Percent Female!!MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Population 25 years and over with earnings!!Graduate or professional degree
0,0400000US01,Alabama,457530,56454,158761,207319,34996,3360058,130320,302753,...,33.6,18.2,12.7,5.0,(X),(X),(X),(X),(X),(X)
1,0400000US02,Alaska,67738,8447,27173,28997,3121,484058,10424,20666,...,25.9,15.2,8.2,3.4,(X),(X),(X),(X),(X),(X)
2,0400000US04,Arizona,694529,100988,246748,276860,69933,4944540,252459,360539,...,26.7,15.1,11.0,5.1,(X),(X),(X),(X),(X),(X)
3,0400000US05,Arkansas,282336,33273,97402,126196,25465,2036456,94376,160617,...,30.9,17.6,14.1,5.3,(X),(X),(X),(X),(X),(X)
4,0400000US06,California,3683287,367782,1151214,1736881,427410,26937872,2346235,1955278,...,22.4,14.1,10.4,4.8,(X),(X),(X),(X),(X),(X)
5,0400000US08,Colorado,527473,65561,167955,221559,72398,3974943,127702,174518,...,21.0,12.9,9.1,4.0,(X),(X),(X),(X),(X),(X)
6,0400000US09,Connecticut,341435,35160,102175,147669,56431,2496420,99132,132707,...,27.0,13.2,9.5,3.8,(X),(X),(X),(X),(X),(X)
7,0400000US10,Delaware,82507,9213,26864,38168,8262,687311,23925,42597,...,28.6,15.0,9.2,3.9,(X),(X),(X),(X),(X),(X)
8,0400000US11,District of Columbia,72703,3199,19034,33665,16805,505145,16459,24709,...,38.3,21.9,18.4,4.3,(X),(X),(X),(X),(X),(X)
9,0400000US12,Florida,1761280,252646,565965,755137,187532,15484502,712764,1081871,...,26.8,15.4,10.6,5.7,(X),(X),(X),(X),(X),(X)


In [15]:
edu_columns = ["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years","Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher","Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over","Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher","Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher"]

#edu_columns = ["Geographic Area Name", "id", "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT"]

edu_columns

['Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years',
 "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher",
 'Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over',
 "Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher",
 "Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher"]

In [16]:
educationData = educationData[edu_columns]
educationData

,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over,Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher,Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher
0,457530,34996,3360058,885357,28.5
1,67738,3121,484058,146157,35.7
2,694529,69933,4944540,1492158,31.1
3,282336,25465,2036456,475367,24.3
4,3683287,427410,26937872,9428484,35.6
5,527473,72398,3974943,1695602,44.3
6,341435,56431,2496420,994548,42.6
7,82507,8262,687311,228199,34.6
8,72703,16805,505145,301429,89.3
9,1761280,187532,15484502,4753637,32.5


In [17]:
educationData['total_with_college'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]

<ipython-input-17-3f59ff08e8b6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  educationData['total_with_college'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years!!Bachelor's degree or higher"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree or higher"]


In [18]:
educationData['total_pop'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over"]

<ipython-input-18-2f60fb37d420>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  educationData['total_pop'] = educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 18 to 24 years"] + educationData["Estimate!!Total!!AGE BY EDUCATIONAL ATTAINMENT!!Population 25 years and over"]


In [19]:
educationData['percent_with_degree'] = educationData['total_with_college'] / educationData['total_pop'] * 100

<ipython-input-19-a1aa71eb4d15>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  educationData['percent_with_degree'] = educationData['total_with_college'] / educationData['total_pop'] * 100


In [20]:
edu_feats = ["percent_with_degree", "Estimate!!Percent!!RACE AND HISPANIC OR LATINO ORIGIN BY EDUCATIONAL ATTAINMENT!!White alone!!Bachelor's degree or higher"]

In [21]:
eduFeatData = educationData[edu_feats]

demoFeatData

In [22]:
demofeat_array = demoFeatData.to_numpy()

In [23]:
edufeat_array = eduFeatData.to_numpy()

In [24]:
#rural

In [25]:
#religion

In [80]:
#unemployment
unemploymentData = pd.read_csv("Data/State unemployment - Sheet1.csv")

unemploymentData = unemploymentData.sort_values(by = "State")
unemploymentData = unemploymentData["Unemployment Percentage"]

In [81]:
unemployment_array = unemploymentData.to_numpy()
unemployment_array

array([ 6.6,  7.2,  6.7,  7.3, 11. ,  6.4,  7.8,  8.2,  8.7,  7.6,  6.4,
       15.1,  6.1, 10.2,  6.2,  4.7,  5.9,  5.6,  8.1,  6.1,  7.2,  9.6,
        8.5,  6. ,  7.1,  4.9,  5.3,  3.5, 12.6,  6. ,  6.7,  9.4,  9.7,
        7.3,  4.4,  8.4,  5.3,  8. ,  8.1, 10.5,  5.1,  4.1,  6.3,  8.3,
        5. ,  4.2,  6.2,  7.8,  8.6,  5.4,  6.1])

In [82]:
feat_array = np.concatenate((demofeat_array[0:51],edufeat_array[0:51],unemployment_array[:,np.newaxis]), axis = 1)

In [83]:
feat_array

array([[ 3.94000000e+01,  2.69000000e+01,  5.00000000e-01,
         1.30000000e+00,  4.50000000e+00,  3.32588817e+01,
        -8.68295337e+01,  2.41082327e+01,  2.85000000e+01,
         6.60000000e+00],
       [ 3.50000000e+01,  3.10000000e+00,  1.58000000e+01,
         6.00000000e+00,  7.20000000e+00,  6.44459613e+01,
        -1.49680909e+02,  2.70531138e+01,  3.57000000e+01,
         7.20000000e+00],
       [ 3.83000000e+01,  4.70000000e+00,  4.60000000e+00,
         3.30000000e+00,  3.17000000e+01,  3.43953420e+01,
        -1.11763276e+02,  2.77012216e+01,  3.11000000e+01,
         6.70000000e+00],
       [ 3.88000000e+01,  1.55000000e+01,  6.00000000e-01,
         1.50000000e+00,  7.70000000e+00,  3.52048883e+01,
        -9.24479108e+01,  2.15988325e+01,  2.43000000e+01,
         7.30000000e+00],
       [ 3.70000000e+01,  5.80000000e+00,  8.00000000e-01,
         1.48000000e+01,  3.94000000e+01,  3.67014631e+01,
        -1.18755997e+02,  3.21865479e+01,  3.56000000e+01,
         1.

In [74]:
#feat_array = 

In [84]:
nbrs = NearestNeighbors().fit(feat_array)

In [85]:
distances, indices = nbrs.kneighbors(feat_array)

In [86]:
indices

array([[ 0, 18, 40, 42, 24],
       [ 1, 37, 12, 26, 44],
       [ 2, 28, 43,  4, 31],
       [ 3, 42, 14, 25, 17],
       [ 4,  2, 28, 43, 31],
       [ 5, 44, 16, 27,  2],
       [ 6, 32, 30, 21, 39],
       [ 7, 33, 40, 46, 38],
       [ 8, 20, 46, 21, 32],
       [ 9, 13, 33, 32, 30],
       [10, 40, 33,  0, 18],
       [11, 37,  4,  1, 28],
       [12, 50, 44, 37, 26],
       [13, 16, 49, 22, 46],
       [14, 17, 35, 22, 25],
       [15, 49, 25, 14, 27],
       [16, 27, 15, 23, 49],
       [17, 14, 35, 48, 42],
       [18, 24,  0, 10, 40],
       [19, 29, 45, 38, 39],
       [20, 46,  7, 10, 33],
       [21,  6, 30, 32, 47],
       [22, 35, 14, 38, 42],
       [23, 49, 27, 16, 15],
       [24, 18,  0, 10, 40],
       [25, 15, 49, 14, 42],
       [26, 50, 34, 41, 12],
       [27, 16, 15, 23, 34],
       [28,  2,  4, 12, 37],
       [29, 45, 19, 38, 47],
       [30, 32,  6, 21, 39],
       [31, 43,  2,  4, 28],
       [32, 30,  6, 46, 39],
       [33,  7, 40, 46, 42],
       [34, 41

In [87]:
demographicData['Abbreviation'] = demographicData["Geographic Area Name"].map(us.states.mapping('name', 'abbr'))

In [88]:
for state in indices:
    print(demographicData.iloc[state[0]]['Geographic Area Name'] + ': ' + demographicData.iloc[state[1]]['Geographic Area Name']
         + ', ' + demographicData.iloc[state[2]]['Geographic Area Name'] + ', ' + demographicData.iloc[state[3]]['Geographic Area Name'] + ', ' 
          + demographicData.iloc[state[4]]['Geographic Area Name'])

Alabama: Louisiana, South Carolina, Tennessee, Mississippi
Alaska: Oregon, Idaho, Montana, Utah
Arizona: Nevada, Texas, California, New Mexico
Arkansas: Tennessee, Indiana, Missouri, Kentucky
California: Arizona, Nevada, Texas, New Mexico
Colorado: Utah, Kansas, Nebraska, Arizona
Connecticut: New York, New Jersey, Massachusetts, Rhode Island
Delaware: North Carolina, South Carolina, Virginia, Pennsylvania
District of Columbia: Maryland, Virginia, Massachusetts, New York
Florida: Illinois, North Carolina, New York, New Jersey
Georgia: South Carolina, North Carolina, Alabama, Louisiana
Hawaii: Oregon, California, Alaska, Nevada
Idaho: Wyoming, Utah, Oregon, Montana
Illinois: Kansas, Wisconsin, Michigan, Virginia
Indiana: Kentucky, Ohio, Michigan, Missouri
Iowa: Wisconsin, Missouri, Indiana, Nebraska
Kansas: Nebraska, Iowa, Minnesota, Wisconsin
Kentucky: Indiana, Ohio, West Virginia, Tennessee
Louisiana: Mississippi, Alabama, Georgia, South Carolina
Maine: New Hampshire, Vermont, Pennsylv